In [1]:
import plotly.express as px
import geopandas as gpd
from fetch_data import make_dict_from_cols
import os
import json
import shapely
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [2]:
geo_stations = gpd.read_file('./data/geojson/Amtrak_Project_Stations.geojson')
geo_stations

,OBJECTID,STNCODE,STNNAME,CITY2,STATE,STFIPS,urban,geometry
0,1,NYP,"New York (Penn Station), New York",New York,NY,36,YES,POINT (-73.99446 40.75033)
1,2,WAS,"Washington, District of Columbia",Washington,DC,11,YES,POINT (-77.00642 38.89699)
2,3,PHL,"Philadelphia (30th St), Pennsylvania",Philadelphia,PA,42,YES,POINT (-75.18104 39.95562)
3,5,BOS,"Boston (South Station), Massachusetts",Boston,MA,25,YES,POINT (-71.05530 42.35231)
4,8,BAL,"Baltimore (Penn Station), Maryland",Baltimore,MD,24,YES,POINT (-76.61569 39.30730)
5,10,PVD,"Providence, Rhode Island",Providence,RI,44,YES,POINT (-71.41348 41.82949)
6,12,WIL,"Wilmington, Delaware",Wilmington,DE,10,YES,POINT (-75.55110 39.73726)
7,13,BWI,"BWI Rail Station at Thurgood Marshall Airport,...",BWI Marshall Airport,MD,24,YES,POINT (-76.69430 39.19236)
8,14,NWK,"Newark (Penn Station), New Jersey",Newark,NJ,34,YES,POINT (-74.16475 40.73471)
9,16,NHV,"New Haven, Connecticut",New Haven,CT,09,YES,POINT (-72.92667 41.29771)


In [3]:
amtrak_stations = ['BOS', 'BBY', 'RTE', 'PVD', 'KIN', 'NLC', 'NHV', 'STM',
                   'NYP', 'NWK', 'TRE', 'PHL', 'WIL', 'BAL', 'BWI', 'NCR', 'WAS']

mile_markers = {'Northbound Mile': {station: None for station in amtrak_stations} , 'Southbound Mile': {station: None for station in amtrak_stations} } 

SB = [0,1,11,43,70,105,156,195,231,241,289,322,347,416,427,448,457]

NB = [457, 456, 446, 414,387,352,301,262,226,216,168,135,110,41,30,9,0]

for station, NB_mile, SB_mile in zip(amtrak_stations, NB, SB):
    mile_markers['Northbound Mile'][station] = NB_mile
    mile_markers['Southbound Mile'][station] = SB_mile


mile_cols = pd.DataFrame.from_dict(mile_markers, orient='columns')
mile_cols.to_csv('./data/geojson/mile_markers.csv', index=False)

In [5]:
for i in range(1,len(SB)):
    print(SB[i]-SB[i-1], amtrak_stations[i], amtrak_stations[i-1])

1 BBY BOS
10 RTE BBY
32 PVD RTE
27 KIN PVD
35 NLC KIN
51 NHV NLC
39 STM NHV
36 NYP STM
10 NWK NYP
48 TRE NWK
33 PHL TRE
25 WIL PHL
69 BAL WIL
11 BWI BAL
21 NCR BWI
9 WAS NCR


In [4]:
gdf = pd.DataFrame(geo_stations)

gdf = gdf.set_index(gdf['STNCODE'])

def get_new_index(stncode_col):
    new_index = []
    for i in range(len(amtrak_stations)):
        for j in range(gdf.shape[0]):
            if amtrak_stations[i] == gdf['STNCODE'][j]:
                new_index.append(j)
    return new_index
            
new_index = get_new_index(gdf['STNCODE'])
df = gdf.join(mile_cols, how='inner')
df = df.iloc[new_index].reindex()
df.reset_index(drop=True)

,OBJECTID,STNCODE,STNNAME,CITY2,STATE,STFIPS,urban,geometry,Northbound Mile,Southbound Mile
0,5,BOS,"Boston (South Station), Massachusetts",Boston,MA,25,YES,POINT (-71.05530 42.35231),457,0
1,17,BBY,"Boston (Back Bay), Massachusetts",Boston,MA,25,YES,POINT (-71.07583 42.34732),456,1
2,24,RTE,"Westwood, Route 128 Station, Massachusetts",Route 128,MA,25,YES,POINT (-71.14789 42.21024),446,11
3,10,PVD,"Providence, Rhode Island",Providence,RI,44,YES,POINT (-71.41348 41.82949),414,43
4,60,KIN,"West Kingston, Rhode Island",Kingston,RI,44,,POINT (-71.56060 41.48396),387,70
5,63,NLC,"New London, Connecticut",New London,CT,09,YES,POINT (-72.09322 41.35427),352,105
6,16,NHV,"New Haven, Connecticut",New Haven,CT,09,YES,POINT (-72.92667 41.29771),301,156
7,27,STM,"Stamford, Connecticut",Stamford,CT,09,YES,POINT (-73.54216 41.04713),262,195
8,1,NYP,"New York (Penn Station), New York",New York,NY,36,YES,POINT (-73.99446 40.75033),226,231
9,14,NWK,"Newark (Penn Station), New Jersey",Newark,NJ,34,YES,POINT (-74.16475 40.73471),216,241


In [5]:
mapbox_token = os.environ.get('MAPBOX_TOKEN')
assert mapbox_token is not None, 'empty token'

In [7]:
px.set_mapbox_access_token(mapbox_token)
fig = px.scatter_mapbox(geo_stations,
                        lat=geo_stations.geometry.y,
                        lon=geo_stations.geometry.x,
                        mapbox_style = 'mapbox://styles/elizabethchen/ckhwyiwxw09o11ap9y8rpvldf',
                        hover_name="STNNAME",
                        center=dict(
                            lat=40.58,
                            lon=-74.00
                        ),
                        zoom=5.5,
                        )

config = dict({'scrollZoom': False})
fig.show(config=config)

In [8]:
geo_df = gpd.read_file('./data/geojson/Amtrak_Project_Routes.geojson')

lats = []
lons = []
names = []

for feature in geo_df.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
fig = px.line_mapbox(lat=lats, lon=lons,
                     mapbox_style='mapbox://styles/elizabethchen/ckhwyiwxw09o11ap9y8rpvldf', 
                     zoom=5.5
        )
fig.show()

In [9]:
lats = []
lons = []
for feature in geo_df.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        
print(np.min(lats), np.max(lats))
print(np.min(lons), np.max(lons))

37.023023966194216 42.347551329197486
-77.50759063408643 -71.07514940391806
